In [ ]:
import glob
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import matplotlib
import random
import datetime
import time

Load data from each cage and combine it into a pandas dataframe. This must be done manually by uncommenting each cage number one by one. (I know, it's stupid, but its only needed once)

In [ ]:
folders = ['15.6'] #,'15.1', '15.2', '15.3', '15.4', '15.5', '15.6']
cages = []
path = "/path/to/data/"
for folder in folders:#
    print("Looking at: " + path+folder)
    cages.append((sorted(glob.glob(path+folder+'/*biomass.csv'))))

print(len(cages))
cage_dfs = []
for i, cage in enumerate(cages):
    if i > 3:
        break
    
    li = []
    for file in cage:
        df = pd.read_csv(file, index_col=None, header=None)#,error_bad_lines=False)
        print(df.shape)
        li.append(df)

    cage_dfs.append(pd.concat(li, axis=0, ignore_index=True))
dataframe = cage_dfs[0]


Use dates in dataframe to set disease onset date

In [ ]:
targets = np.zeros((dataframe.shape[0], 1))
sick_time = datetime.datetime(*time.strptime('2019-03-15', '%Y-%m-%d')[:6])
print(targets.shape)
print(dataframe.shape)


In [ ]:
for i in range(dataframe.shape[0]):
    if dataframe[3][i] != 'date_time':
        if datetime.datetime(*time.strptime(dataframe[3][i][0:10], '%Y-%m-%d')[:6]) >= sick_time:
            targets[i] = 1.0

Remove date and surpurflous information to only be left with a clean data dataframe

In [ ]:
dataframe = dataframe.drop([0, 1, 2, 3], axis=1)

Only a small portion of the echo data is actually from below the water surface. Remove all air from data as well as noisy lower part.

In [ ]:
np_data = dataframe.values
np_data = np_data[:, 40:232]
np_data.shape

Plot data if you wish

In [ ]:
length = 1000
def plot_data(dataset, max_value=0):
    num_iter = dataset.shape[0]//length
    print(num_iter)
    prev = 0
    for i in range(10):
        print((i+1)*length)
        show = dataset[prev:(i+1)*length]
        #print(show.shape)
        #show = show[:,14:246]
        #print(show.shape)
        base = 20
        show = np.log(show) / np.log(base)  # = [3, 4]
        show[np.isneginf(show)] = 0
        prev = (i+1)*length
        show = (show-np.max(show))
        show = np.rot90(show, 1)
        font = {'family' : 'normal',
        'size'   : 40}

        matplotlib.rc('font', **font)
        #plt.axis('off')
        matplotlib.rcParams['figure.figsize'] = [35, 15]
        plt.imshow(show, cmap="gnuplot2")
        plt.title("Echogram from feeding zone")
        #plt.colorbar()
        plt.show()


In [ ]:
plot_data(np_data)

Save your new dataframe into a .hdf5 data file.

In [ ]:
def save_data(dataset, path, labels=""):
    save_path = path+folders[0]+"_whole_cage_all_dates"+labels+".hdf5"
    print(save_path)
    with h5py.File(save_path, "w") as f:
            dset = f.create_dataset("mydataset", data=dataset)

In [ ]:
save_data(np_data, '/path/to/save/data/', '_data')
save_data(targets, '/path/to/save/targets/', '_targets')